In [1]:
# Import the requests library.
import requests
import numpy as np
import pandas as pd
# Import the requests library.
import requests
import datetime
from datetime import datetime
import matplotlib.pyplot as plt
# Dependencies for the wine API
import urllib
import json
# Import the API key.
from config import weather_api_key
import calendar

Token = 'tylRGHbxqiZUDNymMowCBFCuEqeULiWk'
API_Token='Token d8f94b93417333267ca0edcca7dc14a9e035e1bb'

In [2]:
# Import the API key.
headers = {
    'accept': 'application/json',
    'authorization': API_Token
    }

# Red Wine in USA API

In [3]:
#import JSON data with the API
req_red_USA = urllib.request.Request('https://api.globalwinescore.com/globalwinescores/latest/?color=red&limit=5000&country=Usa', data=None, headers=headers, origin_req_host=None, unverifiable=False, method=None)
res_red_USA = urllib.request.urlopen(req_red_USA).read().decode()
obj_red_USA = json.loads(res_red_USA)
#read json into DF
red_USA_df = pd.DataFrame.from_dict(obj_red_USA['results'], orient='columns')
# Delete unnecessary columns
red_USA_df.drop([ 'wine_slug', 'appellation_slug',  'date',  'lwin', 'lwin_11'],axis=1, inplace=True)
##'wine_id','wine_type','classification',
##Drop NA values
Red_USA_df = Red_USA_df.dropna()
#convert vintage to date type
red_USA_df['vintage']= pd.to_datetime(red_USA_df['vintage']).dt.year
red_USA_df['regions']=red_USA_df['regions'].str.get(0)
# Read the DF to a csv
red_USA_df.to_csv("Red_Wine_USA.csv")

NameError: name 'Red_USA_df' is not defined

# White Wine in USA API

In [ ]:
#import JSON data with the API
req_white_USA = urllib.request.Request('https://api.globalwinescore.com/globalwinescores/latest/?color=white&limit=5000&country=Usa', data=None, headers=headers, origin_req_host=None, unverifiable=False, method=None)
res_white_USA = urllib.request.urlopen(req_white_USA).read().decode()
obj_white_USA = json.loads(res_white_USA)
#read json into DF
white_USA_df = pd.DataFrame.from_dict(obj_white_USA['results'], orient='columns')
# Delete unnecessary columns
white_USA_df.drop([ 'wine_slug', 'appellation_slug',  'date',  'lwin', 'lwin_11'],axis=1, inplace=True)
##'wine_id','wine_type','classification',
##Drop NA values
white_USA_df = white_USA_df.dropna()
#convert vintage to date type
white_USA_df['vintage']= pd.to_datetime(white_USA_df['vintage']).dt.year
white_USA_df['regions']=white_USA_df['regions'].str.get(0)
# Read the DF to a csv
white_USA_df.to_csv("White_Wine_USA.csv")

# Weather

# California Avg Temp and Precipitation API

In [ ]:
#Create average temperature

# Create empty lists to append the data into
dates_temp = []
temps = []
dates_prcp = []
prcp = []

#Identify what years we want Temp and Prcp data for
for year in range(2013, 2017):
    year = str(year)
    #Print out 'working on year' to idenfity if script is running correctly
    print('working on year '+year)
    
    #make the api call for temp and precipitation
    r = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TAVG&limit=1000&stationid=GHCND:USW00023129&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})
    p = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&limit=1000&stationid=GHCND:USC00046074&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})
    #Load JSON data
    d = json.loads(r.text)
    e = json.loads(p.text)
    #Get TAVG and PRCP data
    avg_temps = [item for item in d['results'] if item['datatype']=='TAVG']
    avg_prcp = [item for item in e['results'] if item['datatype']=='PRCP']
    #get the date field from all average temperature readings
    dates_temp += [item['date'] for item in avg_temps]
    dates_prcp += [item['date'] for item in avg_prcp]
    #get the actual average temperature from all average temperature readings
    temps += [item['value'] for item in avg_temps]
    prcp += [item['value'] for item in avg_prcp]
    
# initialize dataframe
df_temp = pd.DataFrame()
df_prcp = pd.DataFrame()

# populate date and average temperature fields (cast string date to datetime)
df_temp['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
df_prcp['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_prcp]
# convert to degrees F
df_temp['avgTemp'] = [float(v)/10.0*1.8 + 32 for v in temps]
#convert to mm to inches
df_prcp['avgPrcp'] = [float(v)*0.0393701 for v in prcp]
# Average the temp/prcp per day to temp/prcp per month
df_temp = df_temp.set_index('date').resample('M').mean()
df_prcp = df_prcp.set_index('date').resample('M').mean()
# Combine the series into a df
df_temp['avgPrcp']=df_prcp['avgPrcp']


# Washington Avg Temp and Precipitation API

In [ ]:
#Create average temperature
dates_temp_W = []
temps_W = []
dates_prcp_W = []
prcp_W = []

#for each year from 2015-2019 ...
for year in range(2013, 2017):
    year = str(year)
    print('working on year '+year)
    
    #make the api call for temp and precipitation
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TAVG&limit=1000&stationid=GHCND:USW00023129&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})
    p = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&limit=1000&stationid=GHCND:USC00046074&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})
    #Load JSON data
    d = json.loads(r.text)
    e = json.loads(p.text)
    #Get TAVG and PRCP data
    avg_temps = [item for item in d['results'] if item['datatype']=='TAVG']
    avg_prcp = [item for item in e['results'] if item['datatype']=='PRCP']
    #get the date field from all average temperature readings
    dates_temp_W += [item['date'] for item in avg_temps]
    dates_prcp_W += [item['date'] for item in avg_prcp]
    #get the actual average temperature from all average temperature readings
    temps_W += [item['value'] for item in avg_temps]
    prcp_W  += [item['value'] for item in avg_prcp]

#initialize dataframe
df_temp_W = pd.DataFrame()
df_prcp_W = pd.DataFrame()

#populate date and average temperature fields (cast string date to datetime
df_temp_W['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp_W]
df_prcp_W['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_prcp_W]
#convert to degrees F
df_temp_W['avgTemp'] = [float(v)/10.0*1.8 + 32 for v in temps_W]
#convert to mm to inches
df_prcp_W['avgPrcp'] = [float(v)*0.0393701 for v in prcp_W]
# Average the temp/prcp per day to temp/prcp per month
df_temp_W = df_temp_W.set_index('date').resample('M').mean()
df_prcp_W = df_prcp_W.set_index('date').resample('M').mean()
# Combine the series into a df
df_temp_W['avgPrcp']=df_prcp_W['avgPrcp']


#  Oregon Avg Temp and Precipitation API

In [ ]:
#Create average temperature
dates_temp_O = []
temps_O = []
dates_prcp_O = []
prcp_O = []

#for each year from 2015-2019 ...
for year in range(2013, 2017):
    year = str(year)
    print('working on year '+year)
    
    #make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TAVG&limit=1000&stationid=GHCND:USW00023129&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})
    p = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&limit=1000&stationid=GHCND:USC00046074&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})
    #load the api response as a json
    d = json.loads(r.text)
    e = json.loads(p.text)
    #get all items in the response which are average temperature readings
    avg_temps = [item for item in d['results'] if item['datatype']=='TAVG']
    avg_prcp = [item for item in e['results'] if item['datatype']=='PRCP']
    #get the date field from all average temperature readings
    dates_temp_O += [item['date'] for item in avg_temps]
    dates_prcp_O += [item['date'] for item in avg_prcp]
    #get the actual average temperature from all average temperature readings
    temps_O += [item['value'] for item in avg_temps]
    prcp_O += [item['value'] for item in avg_prcp]

#initialize dataframe
df_temp_O = pd.DataFrame()
df_prcp_O = pd.DataFrame()

#populate date and average temperature fields (cast string date to datetime
df_temp_O['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp_O]
df_prcp_O['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_prcp_O]
#convert to degrees F
df_temp_O['avgTemp'] = [float(v)/10.0*1.8 + 32 for v in temps_O]
#convert to mm to inches
df_prcp_O['avgPrcp'] = [float(v)*0.0393701 for v in prcp_O]
# Average the temp/prcp per day to temp/prcp per month
df_temp_O = df_temp_O.set_index('date').resample('M').mean()
df_prcp_O = df_prcp_O.set_index('date').resample('M').mean()
# Combine the series into a df
df_temp_O['avgPrcp']=df_prcp_O['avgPrcp']

# Add Lat/Long/Elevation

In [ ]:
# create new columns for Lat/Long/Elevation for the red and white white data
#There are three clusters of wineries. 
#The wineries in California, Washington, and Oregon are within a couple dozen miles of one another in each state.
#So I just used three stations: One for California, one for Washington, one for Oregon


#-------------------------------------------------------Station IDs---------------------------------------------------#
#California_GHCND:
Nappa_Valley_g='GHCND:USC00046074'
# Washington_GHCND:
Walla_Walla_Valley_g='GHCND:GHCND:USC00457015'
# Oregon_GHCND:
Willamette_Valley_g='GHCND:USW00024232'
#--------------------------------------------------------Latitude-----------------------------------------------------#

#Latitude
#California_LAT:
Cali_lat='38.2777'
# Oregon_lat:
Oregon_lat='44.905'
# Washington_Lat:
Washington_lat='46.3119'


#Create a new column in red_USA_df that has latitude for california, washingtion, oregon
latitude=[]
for lat in red_USA_df['regions']:
#California
    if lat =='California':
        latitude.append(Cali_lat)
#Oregon
    elif lat == 'Oregon':
        latitude.append(Oregon_lat)
#Washington
    else:
        latitude.append(Washington_lat)
red_USA_df['lat']=latitude

#Create a new column in white_USA_df that has latitude for california, washingtion, oregon
latitude=[]
for lat in white_USA_df['regions']:
#California
    if lat =='California':
        latitude.append(Cali_lat)
#Oregon
    elif lat == 'Oregon':
        latitude.append(Oregon_lat)
#Washington
    else:
        latitude.append(Washington_lat)
white_USA_df['lat']=latitude

#------------------------------------------------------Longitude-------------------------------------------------------------#

#Longitude
#California_Long:
Cali_long='-122.2647'
# Oregon_Long:
Oregon_long='-123.0011'
# Washington_Long:
Washington_long= '-119.2633'

#Create a new column in red_USA_df that has longitude for california, washingtion, oregon
longitude=[]
for long in red_USA_df['regions']:
#California
    if long =='California':
        longitude.append(Cali_long)
#Oregon
    elif long == 'Oregon':
        longitude.append(Oregon_long)
#Washington
    else:
        longitude.append(Washington_long)
red_USA_df['long']=longitude 

#Create a new column in white_USA_df that has longitude for california, washingtion, oregon
longitude=[]
for long in white_USA_df['regions']:
#California
    if long =='California':
        longitude.append(Cali_long)
#Oregon
    elif long == 'Oregon':
        longitude.append(Oregon_long)
#Washington
    else:
        longitude.append(Washington_long)
white_USA_df['long']=longitude 

#-----------------------------------------------------------------Elevation------------------------------------------------#

#Elevation (feet)
#California_Elevation:
Cali_ele='35.1'
# Oregon_Elevation:
Oregon_ele='205.1'
# Washington_Elevation:
Washington_ele= '631.9'


#Create a new column in red_USA_df that has elevation for california, washingtion, oregon
elevation=[]
for ele in red_USA_df['regions']:
#California
    if ele =='California':
        elevation.append(Cali_ele)
#Oregon
    elif ele == 'Oregon':
        elevation.append(Oregon_ele)
#Washington
    else:
        elevation.append(Washington_ele)
red_USA_df['elevation']=elevation

#Create a new column in white_USA_df that has elevation for california, washingtion, oregon
elevation=[]
for ele in white_USA_df['regions']:
#California
    if ele =='California':
        elevation.append(Cali_ele)
#Oregon
    elif ele == 'Oregon':
        elevation.append(Oregon_ele)
#Washington
    else:
        elevation.append(Washington_ele)
white_USA_df['elevation']=elevation


In [ ]:
white_USA_df.head(2)

# Add Temp/Precipitation

In [ ]:
# Split the date into columns for days, months, years and then drop days columns for each of the three states
#-------------------------------------------------------------------------------California----------------------------
df_temp.reset_index(inplace= True )
df_temp['day'] = df_temp['date'].dt.day
df_temp['month'] = df_temp['date'].dt.month
df_temp['year'] = df_temp['date'].dt.year

list_months=[]
for x in df_temp['month']:
    list_months.append(calendar.month_name[x])
df_temp['months']=list_months
df_temp.drop(['date', 'day','month'], axis=1, inplace=True)
#----------------------------------------------------------------------------------Oregon-------------------------------
df_temp_O.reset_index(inplace= True )
df_temp_O['day'] = df_temp_O['date'].dt.day
df_temp_O['month'] = df_temp_O['date'].dt.month
df_temp_O['year'] = df_temp_O['date'].dt.year

list_months=[]
for x in df_temp_O['month']:
    list_months.append(calendar.month_name[x])
df_temp_O['months']=list_months
df_temp_O.drop(['date', 'day','month'], axis=1, inplace=True)
#-----------------------------------------------------------------------------------Washington---------------------
df_temp_W.reset_index(inplace= True )
df_temp_W['day'] = df_temp_W['date'].dt.day
df_temp_W['month'] = df_temp_W['date'].dt.month
df_temp_W['year'] = df_temp_W['date'].dt.year

list_months=[]
for x in df_temp_W['month']:
    list_months.append(calendar.month_name[x])
df_temp_W['months']=list_months
df_temp_W.drop(['date', 'day','month'], axis=1, inplace=True)

In [ ]:
df_temp.head(10)

In [ ]:
white_USA_df.head(2)

In [ ]:
# # .apply() method
# white_USA_df['march']=[]

In [ ]:
# # Trying to add columns for weather data for each month

# list_temp_march=[]
# def monthData (year, month, region, vintage):
#     for u,v,x,y,z in zip(white_USA_df['regions'],white_USA_df['vintage'],df_temp['year'], df_temp['avgTemp'], df_temp['months']):
#         if v==x and v==2016 and u=='California'and z=='March':
#             list_temp_march.append(y)
#     #     elif y==z and y==2016 and x=='Oregon'and v=='April':
#     #         list_temp_march.append(u)
#         else:
#             list_temp_march.append(0)
# white_USA_df['march']=white_USA_df['march'].apply(monthData)
# white_USA_df    

In [ ]:
# sorted_white_USA_df=white_USA_df.sort_values(by=['regions'], ascending=False)
# sorted_white_USA_df.head(50)

Imported dependencies<p>
Used the Global Wine Score API to import data for red and white wine in the USA<p>
Imported weather data for California, Oregon, Washington<p>
Added new columns Latitude, Longitude, and Elevation to red and white wine dataframes<p>
Finally I am trying to add new columns for monthly weather data to the red and white wine dataframes<p>